In [3]:
# MÓDULO 1: Setup inicial
import sys, pathlib
import pandas as pd
import numpy as np
import joblib

# Añadir src al path
PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg
from src import evol_utils as eu
from src.predict_xgb import predict_xgb

# Cargar precios y retornos
df_prices = pd.read_parquet(cfg.DATA / "raw" / "prices.parquet").sort_index().ffill()
df_ret = np.log(df_prices / df_prices.shift(1)).dropna()

# Cargar lista de tickers usada en entrenamiento XGB
df_xgb = joblib.load(cfg.DATA / "processed" / "xgb_data.pkl")
tickers = sorted(df_xgb["ticker"].unique())
df_prices = df_prices[tickers]
df_ret = df_ret[tickers]

print("✅ Precios y retornos listos")
print(f"🟢 {len(tickers)} activos | Fechas: {df_ret.index.min().date()} → {df_ret.index.max().date()}")


✅ Precios y retornos listos
🟢 40 activos | Fechas: 2012-05-21 → 2025-06-26


In [5]:
# MÓDULO 2: Función específica para XGB
def rebalancear_en_fecha_xgb(fecha, df_prices, df_ret, tickers):
    try:
        r_hat = predict_xgb(df_prices, fecha, tickers)
        if r_hat.isna().any():
            raise ValueError("Predicción XGB contiene NaNs")

        idx = df_ret.index.get_loc(fecha)
        ventana = df_ret.iloc[idx - cfg.WINDOW: idx]
        Sigma = ventana.cov().values

        if r_hat.shape[0] != Sigma.shape[0]:
            raise ValueError("Dimensión inconsistente entre r̂ y Σ")

        res = eu.resolver_optimizacion(r_hat.values, Sigma)
        w_star = eu.elegir_w_star(res, r_hat.values, Sigma)

        future = df_ret.iloc[idx : idx + cfg.REBAL_FREQ].values @ w_star
        return {"fecha": fecha, "retorno": future.sum(), "w_star": w_star}
    except Exception as e:
        print(f"⚠️ {fecha.date()} | Error: {e}")
        return None


In [7]:
# MÓDULO 3: Loop principal del backtest
fechas = df_ret.loc[cfg.START_BACKTEST:].index
resultados = []

for i in range(cfg.WINDOW, len(fechas) - cfg.REBAL_FREQ, cfg.REBAL_FREQ):
    fecha = fechas[i]
    out = rebalancear_en_fecha_xgb(fecha, df_prices, df_ret, tickers)
    if out is not None:
        resultados.append(out)
        print(f"✅ {fecha.date()} | Retorno {out['retorno']:.4%}")
    else:
        print(f"⚠️ {fecha.date()} | Resultado nulo")


✅ 2019-03-02 | Retorno 0.0238%


KeyboardInterrupt: 

In [10]:
# MÓDULO 4: Guardar resultados
df_res = pd.DataFrame(resultados).set_index("fecha")
joblib.dump(df_res, cfg.RESULT / "backtest_xgb.pkl")
print("💾 Backtest guardado en:", cfg.RESULT / "backtest_xgb.pkl")

💾 Backtest guardado en: C:\Users\ferra\Documents\TFM\results\backtest_xgb.pkl
